<a href="https://colab.research.google.com/github/Material-Educativo/Tecnicas-heuristicas/blob/main/Calendarizacion_de_trabajos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [ ]:
def generar_datos(n_trabajos, semilla=None):
    """Genera datos aleatorios para el problema de calendarización."""
    if semilla is not None:
        np.random.seed(semilla)

    # Tiempos de procesamiento t_i (entre 1 y 20 unidades)
    tiempos = np.random.randint(1, 21, n_trabajos)

    # Penalizaciones p_i (entre 1 y 10 unidades monetarias por unidad de tiempo)
    penalizaciones = np.random.randint(1, 11, n_trabajos)

    # Matriz de costos de preparación s_ij (entre 5 y 50)
    costos_setup = np.zeros((n_trabajos + 2, n_trabajos + 2))
    for i in range(n_trabajos + 2):
        for j in range(n_trabajos + 2):
            if i != j:
                costos_setup[i, j] = np.random.randint(5, 51)

    return tiempos, penalizaciones, costos_setup

In [ ]:
def evaluar_calendario(calendario, tiempos, penalizaciones, costos_setup):
    """Calcula el costo total (retraso + preparación) de un calendario."""
    costo_retraso = 0
    tiempo_acumulado = 0

    # Cálculo del costo de retraso
    for trabajo in calendario:
        idx = trabajo - 1  # los trabajos van de 1 a N
        costo_retraso += tiempo_acumulado * penalizaciones[idx]
        tiempo_acumulado += tiempos[idx]

    # Cálculo del costo de preparación
    costo_setup = costos_setup[0, calendario[0]]  # desde el trabajo ficticio 0
    for i in range(len(calendario) - 1):
        costo_setup += costos_setup[calendario[i], calendario[i + 1]]
    costo_setup += costos_setup[calendario[-1], len(tiempos) + 1]  # hasta N+1

    costo_total = costo_retraso + costo_setup
    return costo_total, costo_retraso, costo_setup

In [ ]:
def mostrar_calendario(calendario, tiempos, penalizaciones, costos_setup):
    """Dibuja un diagrama de Gantt simple del calendario."""
    costo_total, costo_retraso, costo_setup = evaluar_calendario(
        calendario, tiempos, penalizaciones, costos_setup
    )

    fig, ax = plt.subplots(figsize=(10, 4))
    colores = plt.cm.tab20(np.linspace(0, 1, len(tiempos)))
    tiempo_actual = 0

    for trabajo in calendario:
        idx = trabajo - 1
        duracion = tiempos[idx]

        rect = Rectangle(
            (tiempo_actual, 0), duracion, 1,
            facecolor=colores[idx],
            edgecolor='black'
        )
        ax.add_patch(rect)
        ax.text(tiempo_actual + duracion / 2, 0.5, f'T{trabajo}',
                ha='center', va='center', fontsize=10)
        tiempo_actual += duracion

    ax.set_xlim(0, tiempo_actual)
    ax.set_yticks([])
    ax.set_xlabel("Tiempo")
    ax.set_title(
        f"Calendario de trabajos\n"
        f"Costo total: {costo_total:.0f} "
        f"(Retraso: {costo_retraso:.0f}, Setup: {costo_setup:.0f})"
    )
    plt.show()

In [ ]:
def generar_vecinos(calendario):
    """Genera vecinos intercambiando dos trabajos."""
    vecinos = []
    n = len(calendario)
    for i in range(n):
        for j in range(i + 1, n):
            nuevo = calendario.copy()
            nuevo[i], nuevo[j] = nuevo[j], nuevo[i]
            vecinos.append(nuevo)
    return vecinos

## Veamos un ejemplo

In [ ]:
# Ejemplo con 5 trabajos
tiempos, penalizaciones, costos_setup = generar_datos(5, semilla=42)

# Calendario inicial (orden natural de los trabajos)
calendario = [1, 2, 3, 4, 5]

# Evaluar y mostrar resultados
costo_total, costo_retraso, costo_setup = evaluar_calendario(
    calendario, tiempos, penalizaciones, costos_setup
)
print(f"Costo total = {costo_total}")
print(f"Costo de retraso = {costo_retraso}")
print(f"Costo de setup = {costo_setup}")

# Mostrar diagrama de Gantt
mostrar_calendario(calendario, tiempos, penalizaciones, costos_setup)

## Creamos y probamos un ejemplo aleatorio.

In [ ]:
# Ejemplo con 5 trabajos
n_trabajos = 5
tiempos, penalizaciones, costos_setup = generar_datos(n_trabajos, semilla=42)

# Calendario inicial: ejecutar los trabajos en orden natural
calendario_inicial = list(range(1, n_trabajos + 1))

print("Problema de calendarización con", n_trabajos, "trabajos")
print("Calendario inicial:", calendario_inicial)

# Evaluar el calendario inicial
costo_total, costo_retraso, costo_setup = evaluar_calendario(
    calendario_inicial, tiempos, penalizaciones, costos_setup
)
print(f"Costo total: {costo_total:.0f}")
print(f"  - Costo de retraso: {costo_retraso:.0f}")
print(f"  - Costo de setup: {costo_setup:.0f}")

# Mostrar diagrama de Gantt
mostrar_calendario(calendario_inicial, tiempos, penalizaciones, costos_setup)

## Probamos una búsqueda aleatoria.

In [ ]:
np.random.seed(123)
num_calendarios = 500  # número de soluciones aleatorias a evaluar

mejor_calendario = None
mejor_costo = float('inf')
costos_totales = []

# Calendario base (1, 2, ..., N)
calendario_base = list(range(1, n_trabajos + 1))

for _ in range(num_calendarios):
    # Crear una permutación aleatoria del calendario
    calendario = calendario_base.copy()
    np.random.shuffle(calendario)

    # Calcular el costo total
    costo_total, _, _ = evaluar_calendario(
        calendario, tiempos, penalizaciones, costos_setup
    )

    costos_totales.append(costo_total)

    # Guardar el mejor calendario encontrado
    if costo_total < mejor_costo:
        mejor_costo = costo_total
        mejor_calendario = calendario

print(f"\nResultados de {num_calendarios} calendarios aleatorios:")
print(f"Mejor costo encontrado: {mejor_costo:.0f}")
print(f"Peor costo encontrado: {max(costos_totales):.0f}")
print(f"Promedio: {np.mean(costos_totales):.1f} \pm {np.std(costos_totales):.1f}")
print(f"Mejor calendario: {mejor_calendario}")

## Histograma y diagrama de cajas.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# Histograma de costos totales
ax1.hist(costos_totales, bins=30, color='skyblue',
         edgecolor='black', alpha=0.8)
ax1.axvline(mejor_costo, color='green', linestyle='--',
            label=f'Mejor: {mejor_costo:.0f}')
ax1.axvline(np.mean(costos_totales), color='red', linestyle='--',
            label=f'Promedio: {np.mean(costos_totales):.0f}')
ax1.set_xlabel('Costo total')
ax1.set_ylabel('Frecuencia')
ax1.set_title('Distribución de costos en calendarios aleatorios')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Diagrama de caja (boxplot)
ax2.boxplot(costos_totales, vert=True)
ax2.set_ylabel('Costo total')
ax2.set_title('Resumen estadístico')
ax2.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Mostrar el mejor calendario encontrado
mostrar_calendario(mejor_calendario, tiempos, penalizaciones, costos_setup)